In [60]:
import pandas as pd
import numpy as np
import sklearn.model_selection
import matplotlib.pyplot as plt

In [61]:
df = pd.read_csv("data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [62]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2)

In [63]:
training_features = np.array([train["radius_mean"], train["perimeter_mean"], train["area_mean"]]).T
training_diagnosis = train["diagnosis"].T

In [64]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'lbfgs').fit(training_features, training_diagnosis)

In [65]:
training_accuracy = sum(model.predict(training_features) == training_diagnosis.T) / len(training_diagnosis)
training_accuracy

0.8835164835164835

In [66]:
test_features = np.array([test["radius_mean"], test["perimeter_mean"], test["area_mean"]]).T
test_diagnosis = test["diagnosis"]

In [67]:
test_accuracy = sum(model.predict(test_features) == test_diagnosis) / len(test_diagnosis)
test_accuracy

0.956140350877193

In [68]:
df = pd.read_csv("cars.csv")
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [69]:
train, test = train_test_split(df, test_size = 0.2)

In [70]:
features = pd.get_dummies(train["enginelocation"]).reset_index().drop(columns = ["index"])
features.head()

,front,rear
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [71]:
def feature_selector(df, y, num_features):
    possible_features = df.columns.tolist()
    correlation_list = []
    for i in possible_features:
        try:
            coeff = np.corrcoef(df[i], y)[0,1]
        except:
            coeff = 0
        correlation_list.append(coeff)
    correlation_list = [0 if np.isnan(i) else i for i in correlation_list]
    selected_indexes = sorted(range(len(correlation_list)), key = lambda index: abs(correlation_list[index]))[-num_features:]
    return df.columns[selected_indexes]

In [72]:
f = feature_selector(train.drop(columns = ["price"]), train["price"], 5)
f = f.tolist()

In [73]:
features = pd.concat([features, train[f].reset_index()], axis=1).drop(columns= "index")
features.head()

,front,rear,carlength,carwidth,horsepower,curbweight,enginesize
0,1,0,186.7,68.4,95,3075,120
1,1,0,186.6,66.5,110,2758,121
2,1,0,155.9,63.6,70,1874,90
3,1,0,155.9,63.6,70,1909,90
4,1,0,157.3,63.8,68,1989,90


In [74]:
def get_test_feature_matrix(df, num_features):
    dummy = pd.get_dummies(df["enginelocation"]).reset_index().drop(columns = ["index"])
    if "front" not in dummy.columns:
        dummy["front"] = 0
    if "rear" not in dummy.columns:
        dummy["rear"] = 0
    numerical_features = feature_selector(train.drop(columns = ["price"]), train["price"], num_features)
    numerical_features = numerical_features.tolist()
    features = pd.concat([dummy, df[numerical_features].reset_index()], axis=1).drop(columns= "index")
    return features
    

In [75]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(features, train["price"]);

In [76]:
model.coef_

array([-6.93822486e+03,  6.93822486e+03, -2.05686321e+01,  9.05664077e+02,
        2.80424619e+01,  2.81825870e+00,  8.18376644e+01])

In [77]:
from sklearn.linear_model import RidgeCV

ridge_model = RidgeCV()
ridge_model.fit(features, train["price"]);

In [78]:
ridge_model.coef_

array([-6.72869524e+03,  6.72869524e+03, -2.06627730e+01,  8.98515886e+02,
        2.86212567e+01,  2.80173412e+00,  8.20637494e+01])

In [79]:
ridge_model.score(get_test_feature_matrix(test, 5), np.array(test["price"]))

0.8310573056791248

In [80]:
model.score(get_test_feature_matrix(test, 5), np.array(test["price"]))

0.8308093921187477

In [81]:
ridge_model.score(get_test_feature_matrix(train, 5), np.array(train["price"]))

0.8539057176418543

In [59]:
model.score(get_test_feature_matrix(train, 5), np.array(train["price"]))

0.8686135149488405